# Whole declaration embedding

We ditch the chunking as we cannot get a satisfiying match.

## install libs

In [ ]:
!pip install datasets
!pip install langchain
!pip install langchain-groq
# !pip install -U sentence-transformers #check that session is in GPU mode #switch to openAI for a quick fix
!pip install openai
!pip install tiktoken # to count the tokens
!pip install faiss-cpu

## dataset creation (needs GPU)

In [ ]:
dataset_creation = True

In [ ]:
from datasets import load_dataset

if dataset_creation:
  declaration_ds = load_dataset("the-french-artist/hatvp_declarations_xml_plus_markdown", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
if dataset_creation:
  embed_df = declaration_ds.to_pandas()

In [ ]:
# embed_df.head()

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def truncate_text_to_stay_under_openai_embedding_limit(input_text):
  openai_embed_limit = 8192
  delta = num_tokens_from_string(input_text, "cl100k_base") - openai_embed_limit
  while delta > 0:
    input_text = input_text[:-int(delta*2)] #we add factor 2 to speed up the process
    delta = num_tokens_from_string(input_text, "cl100k_base") - openai_embed_limit

  return input_text

In [ ]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   text = truncate_text_to_stay_under_openai_embedding_limit(text)
   return client.embeddings.create(input = [text], model=model).data[0].embedding

get_embedding('this is a test')

In [ ]:
embed_df['markdown_embedding'] = embed_df.apply(lambda row: get_embedding(row['declaration_markdown']), axis=1)

In [ ]:
embed_df

,xml_sha1,declaration_xml,markdown_sha1,declaration_markdown,markdown_embedding
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,7897b9a3c34ee2dd9f6d743520f8ad78363d39cb,# Declaration Information\n\n## General Inform...,"[-0.002353898249566555, 0.02060830034315586, 0..."
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,b0c12e2f087cc8c98b47d5440077899ee9cbb699,# Declaration Information\n\n## General Inform...,"[-0.0038406257517635822, 0.015465394593775272,..."
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,bc15183f3557c5fa34ea637a1e86cfafb13e3f9d,# Declaration Information\n\n## General Inform...,"[-0.005234577227383852, 0.022220566868782043, ..."
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,c71569daa76d9bfb929ab29b1cb57debaf7529bc,# Declaration Information\n\n## General Inform...,"[0.0029081164393574, 0.026618845760822296, 0.0..."
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,47c4c7aeeed69bd771964751aac442d3417c0baf,# Declaration Information\n\n## General Inform...,"[-0.014009842649102211, 0.026249704882502556, ..."
...,...,...,...,...,...
10939,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>15/02/2022 11:07:33</d...,9568f85929aafd4d121b174493644f9f0ed78f42,# Declaration Information\n\n## General Inform...,"[0.008259780704975128, 0.04053126648068428, 0...."
10940,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>17/06/2021 22:39:18</d...,11dae2fec12b37456c4722bd52fd491b9162fbfd,# Declaration Information\n\n## General Inform...,"[0.0053054229356348515, 0.025841478258371353, ..."
10941,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/12/2020 23:48:29</d...,fd57ac8a73f65c63f45dd229ccacdc1e65f2c8b2,# Declaration Information\n\n## General Inform...,"[-0.01021560374647379, 0.05917070806026459, 0...."
10942,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>25/06/2021 13:29:08</d...,1fd74aefa2e4297066fea81bd266bf9597e4ecb5,# Declaration Information\n\n## General Inform...,"[0.018089286983013153, 0.03519130498170853, 0...."


In [ ]:
# %%time
# from sentence_transformers import SentenceTransformer, util

# if dataset_creation:
#   #Load the model
#   # model = SentenceTransformer("intfloat/e5-mistral-7b-instruct") # NEEDS 26+GB VRAM WHICH GOOGLES WONT DELIVER.
#   model = SentenceTransformer("Alibaba-NLP/gte-Qwen1.5-7B-instruct") # STILL TOO MUCH VRAM NEEDED HERE!!!
#   model = SentenceTransformer("intfloat/multilingual-e5-large-instruct") #fingers crossed... NOOPE!


#   doc_emb = model.encode(embed_df.declaration_markdown.to_list())

In [ ]:
# if dataset_creation:
#   embed_df['markdown_embedding'] = list(doc_emb)

In [ ]:
from datasets import Dataset

if dataset_creation:
  embed_ds = Dataset.from_pandas(embed_df)

In [ ]:
embed_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'markdown_sha1', 'declaration_markdown', 'markdown_embedding'],
    num_rows: 10944
})

In [ ]:
if dataset_creation:
  embed_ds.push_to_hub("the-french-artist/hatvp_declarations_markdown_embeds")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/490 [00:00<?, ?B/s]

## dataset loading (no GPU needed)

In [ ]:
from datasets import load_dataset
embed_ds = load_dataset("the-french-artist/hatvp_declarations_markdown_embeds", split='train')

Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
# model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

NameError: name 'SentenceTransformer' is not defined

In [ ]:
embed_ds.add_faiss_index(column='markdown_embedding')

  0%|          | 0/11 [00:00<?, ?it/s]

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'markdown_sha1', 'declaration_markdown', 'markdown_embedding'],
    num_rows: 10944
})

In [ ]:
import numpy as np

def perform_query(query, n_samples=1):
  # query_embed = model.encode([query])
  query_embed = np.array(get_embedding(query))
  scores, retrieved_examples = embed_ds.get_nearest_examples('markdown_embedding', query_embed, k=n_samples)
  return retrieved_examples['declaration_markdown']

In [ ]:
embed_df = embed_ds.to_pandas()

In [ ]:
# potential_true_candidates = embed_df[embed_df['declaration_markdown'].str.contains('ABAD')].declaration_markdown.to_list()
# for curr_candidate in potential_true_candidates:
#   display_markdown(curr_candidate, raw=True)

In [ ]:
from IPython.display import display_markdown

results = perform_query("Qui est Damien Abad ?")

for result in results:
  display_markdown(
      result,
      raw=True)

# Declaration Information

## General Information
- **Type of Declaration ID:** DIA, **Label:** Déclaration d'intérêts et d'activités
- **Label Mandate:** Député ou sénateur, **Type of Mandate:** Député
- **Cod Category Mandate:** PAR, **Name Category Mandate:** Député ou sénateur
- **Cod Type Mandate File:** depute, **Label Type Mandate:** Député
- **Label Organ:** Ain(01), **Code List Organ:** ZEP
- **Help Text:** None
- **Organ Code:** ZEP, **Organ Name:** Zones d'élection des parlementaires, **Organ Label:** Ain(01), **Declaration Label:** None
- **Declarant Quality:** None, **Declarant Quality for PDF:** Député/Ain(01)
- **Start Date Mandate:** 19/06/2022, **End Date Mandate:** None, **Last Declaration Date:** None
- **Marital Regime:** None, **Marital Regime Comments:** None
- **Company Name:** None, **Parent Company Name:** None, **Turnover:** None, **Number of Dwellings:** None
- **Declarant Civility:** M., **Declarant Name:** ABAD, **Declarant First Name:** DAMIEN
- **Declarant Email:**             [Données non publiées]        , **Declarant Telephone:** None, **Declarant Date of Birth:** 05/04/1980
- **Declarant Address:**             [Données non publiées]        , **Address Complement:**             [Données non publiées]        , **Postal Code:**             [Données non publiées]        , **City:**             [Données non publiées]        , **Country:**             [Données non publiées]        

## Attached Files
- **File Name:** VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML, **Server File Name:** None

## Professional Activities

## Consultant Activities

## Volunteer Functions

## Elective Mandates

## Spouse's Professional Activities
- **ID:** CREATION, **Label:** None, **Comment:** None, **Spouse Name:**             [Données non publiées]        , **Employer:** CENTRE HOSPITALIER DU HAUT-BUGEY, **Activity Description:** Infirmière

## Directing Participation
- **Participation ID:** CREATION, **Label:** None, **Comment:** Président du SDIS 01. Activité arrêtée depuis ma démission de la présidence du département en juillet 2017, **Conserved:** None, **Company Name:** SDIS 01, **Description:** Service départemental d'incendie et de secours, **Start Date:** 04/2015, **End Date:** 07/2017
  - **Remuneration Type:** Brut, **Year:** [2015, 2016, 2017], **Amount:** [0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Président de l'association AINTOURISME qui développe et promeut la politique touristique du Département de l'Ain., **Conserved:** None, **Company Name:** ASSOCIATION AINTOURISME, **Description:** Président d'Aintourisme qui a pour objet la politique touristique du Département, **Start Date:** 07/2017, **End Date:** None
  - **Remuneration Type:** Net, **Year:** [2017, 2018, 2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Mon élection comme président du groupe LR à l'assemblée nationale date du 6 Novembre 2019. Ce titre n'octroie pas une rémunération supplémentaire., **Conserved:** None, **Company Name:** Assemblée nationale - Groupe Les Républicains -, **Description:** Président du Groupe Les Républicains à l'Assemblée nationale, **Start Date:** 11/2019, **End Date:** 05/2022
  - **Remuneration Type:** Net, **Year:** [2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Promotion du patrimoine culinaire et gastronomique du Département de l'Ain, **Conserved:** None, **Company Name:** ASSOCIATION SAVEURS DE L'AIN, **Description:** Président de l'association Saveurs de l'Ain qui a pour objet de promouvoir la gastronomie de notre Département, **Start Date:** 05/2019, **End Date:** None
  - **Remuneration Type:** Net, **Year:** [2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0]
- **Participation ID:** CREATION, **Label:** None, **Comment:** Association - parti politique Création Décembre 2018 Nous avons changé de noms à plusieurs reprises : d'abord la France des territoires, puis Dans l'Action puis Les Amis de Damien Abad, nouveau nom depuis Septembre 2020., **Conserved:** None, **Company Name:** Association Les Amis de Damien Abad, **Description:** Président de l'association Les Amis de Damien Abad, **Start Date:** 12/2018, **End Date:** None
  - **Remuneration Type:** Net, **Year:** [2018, 2019, 2020, 2021, 2022], **Amount:** [0.0, 0.0, 0.0, 0.0, 0.0]

## Financial Participations
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** ORANGE, **Evaluation:** 877, **Remuneration:** néant, **Capital Held:** None, **Number of Shares:** 83
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** CREDIT AGRICOLE SA, **Evaluation:** 2910, **Remuneration:** néant, **Capital Held:** None, **Number of Shares:** 341
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** AIRBUS, **Evaluation:** 1929, **Remuneration:** NEANT, **Capital Held:** None, **Number of Shares:** 20
- **ID:** CREATION, **Label:** None, **Comment:**         [Données non publiées]    , **Company Name:** L'OREAL, **Evaluation:** 6552, **Remuneration:** NEANT, **Capital Held:** None, **Number of Shares:** 20

## Parliamentary Collaborators
- **ID:** CREATION, **Label:** None, **Comment:** None, **Collaborator Name:** LOPES MAGNUSON, **Employer:** Néant, **Activity Description:** Assistant fonctionnel
- **ID:** CREATION, **Label:** None, **Comment:** None, **Collaborator Name:** FOUGNIES REBECCA, **Employer:** Néant, **Activity Description:** Assistante parlementaire locale

## Observations
- **ID:** CREATION, **Label:** None, **Comment:** None, **Content:** mon équipe de collaborateurs parlementaires sera complétée dans les prochaines semaines


In [ ]:
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
def get_answer_to_question(question, llm_to_use):

  system = """You are an assistant for question-answering tasks.
  Use the following pieces of retrieved context to answer the question.
  If you don't know the answer, just say that you don't know.
  Use three sentences maximum and keep the answer concise.
  """
  human = "{text}"
  prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

  context = perform_query(question)[0]
  actual_prompt = f"""
  Question: {question}
  Context: {context}
  Answer:
  """

  print(context)
  chain = prompt | llm_to_use | StrOutputParser()
  return chain.invoke({"text": actual_prompt})

In [ ]:
mixtral_llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
llm_llama3_70B = ChatGroq(temperature=0, model_name="llama3-70b-8192")

get_answer_to_question("Qui est Damien Abad ? Et quelle est sa rémunération de député en 2019 ?", llm_llama3_70B)

# Declaration Information

## General Information
- **Type of Declaration ID:** DIA, **Label:** Déclaration d'intérêts et d'activités
- **Label Mandate:** Député ou sénateur, **Type of Mandate:** Député
- **Cod Category Mandate:** PAR, **Name Category Mandate:** Député ou sénateur
- **Cod Type Mandate File:** depute, **Label Type Mandate:** Député
- **Label Organ:** Ain(01), **Code List Organ:** ZEP
- **Help Text:** None
- **Organ Code:** ZEP, **Organ Name:** Zones d'élection des parlementaires, **Organ Label:** Ain(01), **Declaration Label:** None
- **Declarant Quality:** None, **Declarant Quality for PDF:** Député/Ain(01)
- **Start Date Mandate:** 19/06/2022, **End Date Mandate:** None, **Last Declaration Date:** None
- **Marital Regime:** None, **Marital Regime Comments:** None
- **Company Name:** None, **Parent Company Name:** None, **Turnover:** None, **Number of Dwellings:** None
- **Declarant Civility:** M., **Declarant Name:** ABAD, **Declarant First Name:** DAMIEN
- **Declaran

'Damien Abad is a French deputy (Député). His remuneration as a deputy in 2019 is not explicitly mentioned in the provided context.'

# CONCLUSION OF THIS TEST:


- the openai embedding system is very slow (1H for 10k embeddings) but at least it works, unlike the open source lighter ones (in french that is).  
EDIT: my bad, we can parallelize...

- we cannot get interesting information because the retrieval query is very different from the actual query. Eg. What is XXX salary in 2019? The retrieval query will be "get me XXX data" while the generative query is the input one.  

- we are missing information from the markdown files: after 3 days of intensive attempts to make an SQL, then simple Markdown file, from the input XML files, we still cannot ensure that some entire sections are not missing due to the fickle nature of the context


## What to do next:  

- find a way to get embeddings faster and cheaper: we need to run the system in a dedicated notebook with enough VRAM, using a quantized version I suppose (given that we do not have enough VRAM, either locally or online)  
- Make a new dataset from the XML base declarations that simply converts XML to JSON => this will make the context lighter while not need to have massive OO classes that are not able to parse the thing, even after writing more than 250 if statements on "None".  
- find a way to execute online : this system will still need a local FAISS index and/or a local embedding for the dataset creation, as well as an acess to an LLM for retrieval => can we get free (or near free) online stuff to perform these operations? (Groq is already a good thing).  